In [12]:
%load_ext autoreload
%autoreload 2
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch_scatter import scatter_add
import uproot
import awkward as ak

import sys
sys.path.append("..")
from dnn.ak_sample_loader import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
input = AkSampleLoader("/grid_mnt/data_cms_upgrade/cuisset/ticlRegression/PionSamples_v2/histo/histo_9376310_0.root")

In [15]:
input.makeDataAk("feat-v2")

<Array [[[3.8, 2.36, ..., 2.84, 0], ...], ...] type='94 * var * 17 * float64'>

In [16]:
input.caloparticles_splitEndcaps

<Array [{time: 13.2, ...}, {...}, ..., {...}] type='20 * caloparticle[time:...'>

In [8]:
# enum :   enum CellType {
#     CE_E_120 = 0,
#     CE_E_200 = 1,
#     CE_E_300 = 2,
#     CE_H_120 = 3,
#     CE_H_200 = 4,
#     CE_H_300 = 5,
#     CE_H_SCINT = 6,
#     EnumSize = 7
#   };


In [18]:
input.energyPerCellType

<Array [[[0, 0.153, ..., 0.141, 0], ...], ...] type='20 * var * 7 * float32'>

## Version without clipping

In [19]:
true_energy = torch.tensor(input.caloparticles_splitEndcaps.regressed_energy)

In [51]:
tsInEvt_idx = torch.tensor(ak.flatten(ak.broadcast_arrays(ak.local_index(input.tracksters_splitEndcaps.raw_energy, axis=0), input.tracksters_splitEndcaps.raw_energy)[0]))
tsInEvt_idx.shape

torch.Size([44])

In [42]:
def akToTensor_noClip(ar:ak.Array):
    return torch.tensor(ak.flatten(ar))

In [54]:
data = torch.stack([akToTensor_noClip(tracksters_splitEndcaps.raw_energy), akToTensor_noClip(input.tracksters_splitEndcaps.barycenter_eta), akToTensor_noClip(input.tracksters_splitEndcaps.barycenter_z)] + [akToTensor_noClip(input.energyPerCellType[:, :, cellType_i]) for cellType_i in range(7)], dim=-1)

In [57]:
data.shape

torch.Size([44, 10])

In [60]:
class NetworkNoClip(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(data.shape[-1], 1, dtype=data.dtype),
        )

    def forward(self, x):
        return torch.squeeze(self.model(x))
net = NetworkNoClip()

In [61]:
net(data)

tensor([ 29.8255,  32.7520,  28.7608,  27.9655,  36.0864,  28.8798,  32.2802,
         25.1754,  33.4956,  26.7288,  28.3129,  25.7363,  27.1157,  27.5983,
         29.3331,  28.9366,  25.8991,  35.6060,  28.1522,  28.6858,  31.3700,
         28.5225,  32.2580, -32.6915, -33.0454, -32.0779, -32.0959, -35.1889,
        -30.3262, -29.3601, -29.6520, -47.7183, -41.3281, -29.7770, -31.6684,
        -29.0842, -32.9314, -31.8502, -34.3254, -29.0414, -29.1650, -36.3884,
        -34.7929, -34.5013], grad_fn=<SqueezeBackward0>)

In [65]:
scatter_add(net(data), tsInEvt_idx)

tensor([  62.5775,   28.7608,   64.0518,   61.1601,   58.6711,   55.0416,
          25.7363,   27.1157,   85.8680,  210.4936,  -32.6915,  -97.2192,
         -35.1889,  -89.3383,  -89.0464,  -90.5297,  -64.7816,  -34.3254,
         -58.2064, -105.6825], grad_fn=<ScatterAddBackward0>)

In [66]:
loss_fct = nn.MSELoss()
loss = loss_fct(scatter_add(net(data), tsInEvt_idx), true_energy)
loss

tensor(7357.2197, grad_fn=<MseLossBackward0>)

In [69]:
scatter_add(data[:, 0], tsInEvt_idx)/true_energy

tensor([0.3364, 0.1629, 0.3169, 0.4494, 0.3576, 0.3785, 0.6750, 0.4717, 0.4358,
        0.4709, 0.8010, 0.6355, 0.8678, 0.3077, 0.6035, 0.3467, 0.5526, 0.7568,
        0.0697, 0.4638])